# 8. Training & Inference 2 - Process

> 이제 정말 본격적으로 학습과 추론 프로세스를 시작합니다.

> 앞서 정의한 데이터셋, 모델, Loss, Optimizer, Metric을 가지고 실제 파이토치가 모델의 파라미터를 업데이트 하는 과정과 방식을 조금 디테일하게 접근해보겠습니다. 각각의 요소들이 하는 일과, 그로 인해 발생할 수 있는 side effect들이 모델을 업데이트하는 과정을 하나하나 보도록 하겠습니다.

> 그리고, 모델을 추론하고 제출하는 것도 간략히 다루겠습니다.

> 마지막으로, Pytorch Lightning이 어떤 것인지 간략히 다루겠습니다.

<br>

## 8.1 Overview

- 학습과 추론 프로세스의 과정을 이해 하는 것이 목표입니다.

<br>

- 학습 과정

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1lC63Bexlc-IKvxmF0JlrXFmoxayHEQSH' width=500/>

- 출처: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

<br>

- 평가 과정

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1xhESp7aXqdfLjlYoLOVqz7LfYs5f7JPd' width=500/>

- 출처: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

<br>


## 8.2 Training Process

- Training 준비
- Training 프로세스 이해
- More: Gradient Accumulation

<br>

### 8.2.1 Training 준비

- 학습 한번 하기 위해 지금 까지 만든 결과물

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1JbAQSS6mU8U-wngUVcf0VKKObsMH5QJG' width=800/>

<br>

### 8.2.2 Training 프로세스의 이해

#### 8.2.2.1 `model.train()`

- 모델을 train 상태로 바꾼다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1dRfw5dSI_vA_GO4pIg2M4G7xLtbgzBKZ' width=800/>

- 출처: https://pytorch.org/docs/stable/generated/torch.nn.Module.html

<br>

#### 8.2.2.2 `optimizer.zero_grad()`

- optimizer 에 이전 배치 때의 grad 가 남아 있는 것을 0으로 초기화해준다.
  - 엄밀하게 말하면 loss 가 gradient 를 갖고 있는 것이다.
- optimizer 는 모델의 파라미터들을 가지고 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1NGf5s31x7eECPDedUbHE0swOYbVdWQgr' width=600/>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1g6bk9pedrAMo-XDd4KD5Ng6LxpletHKm' width=500/>

<br>

#### 8.2.2.3 `loss = criterion(outputs, labels)`

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1Ci_B8yJvB9bscDcRp8NOrYLF0YXrpsGu' width=700/>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1QMk95L01JlcgjstKZT0TIVQtIBYmwcp3' width=500/>

<br>

#### 8.2.2.4 loss 를 마지막으로 chain 생성

- loss 또한 결국은 `nn.Module` 을 상속받은 것으로 `forward` 함수를 갖고 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1dbK6o7mR2wTwwHrBW05xRV_QFTyMj4Pw' width=700/>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1PPtkboujf5-LfQsoPij6vYRTpHYAi27f' width=500/>

<br>

#### 8.2.2.5 loss 의 `grad_fn` chain -> `loss.backward()`

- 디버깅 과정을 통해 파라미터 확인
- `grad_fn` 에서 연결점 발생
  - 모델의 파라미터까지 연결된다.
- `loss.backward()` 를 했을 때 연결된 파라미터들에게 gradient 를 업데이트하게 된다.
- loss 의 역할은 gradient 를 업데이트하는 것이다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1qh0nKasUVchL9YrHDazpHFbOzSPlqOMw' width=500/>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1d2hvxRbFl7KH2tVkVZ4-qXeBZlmNQgcK' width=500/>

<br>

#### 8.2.2.6 `optimizer.step()`

- `loss.backward()` 를 수행했기 때문에 모델의 각 파라미터의 grad 는 업데이트 되어 있다.
- 이 때 `optimizer.step()` 을 하면 optimizer 가 가지고 있는 파라미터의 업데이트된 grad 를 바탕으로 실제 데이터에 적용 시킨다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1eOcmGZwCydwiAqrueeuS2qPhp2uqXcSW' width=500/>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1RupwR4UZ3exXeMBoXlG_9GzmJf8rvUxY' width=500/>

<br>

### 8.2.3 More: Gradient Accumulation

- 지금까지의 모든 과정을 이해했다면 이를 응용하는 것도 가능하다.

- 기존의 training process

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1EipNannV7mgj8zeVN0-ncuPSOVZhIUdy' width=500/>

- `NUM_ACCUM = 2` 로 지정하므로서 배치가 2번 돌 때마다 누적된 gradient 를 한 번에 적용하는 것이 <font color='yellow'>Gradient Accumulation</font> 이다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=177-xOYAAjg0xbtLdiH087XvqXn3JMjk-' width=500/>



<br>

## 8.3 Inference Process

- Inference 프로세스 이해
- Validation
- Checkpoint
- 최종 Output, Submission 형태로 변환

<br>

### 8.3.1 Inference 프로세스 이해

#### 8.3.1.1 `model.eval()`

- model 을 eval 상태로 변경

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1uGLMNMJEyIl0Q0qOn9FTzKSuzfKU3fqC' width=800/>

- 출처: https://pytorch.org/docs/stable/generated/torch.nn.Module.html

<br>

#### 8.3.1.2 `with torch.no_grad():`

- inference 과정에서 gradient 가 업데이트가 안되도록 설정

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1_I1Ny0vOQ6-vtk05o8e66t7l9HSMKSXs' width=500/>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1GVAWfouxIJfbk5llUbModPdTX7PCjIvB' width=500/>

<br>

### 8.3.2 Validation 확인

- 추론 과정에 Validation 셋이 들어가면 그게 검증이다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1UL1f6P-0PtQDmtxIqOX1o5wp8ley0-N9' width=500/>

<br>

### 8.3.3 Checkpoint

- 그냥 직접 짜면 된다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1reKh-uXhIzqlqeROgQl-ulh01jw8V8cZ' width=700/>

<br>

### 8.3.4 최종 Output, Submission 형태로 변환

- 최종 Submissioni 스펙을 확인 후 변환하여 제출

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1JgyxnkGrEwyprY1N6nYN8astAiI9_CKf' width=600/>

<br>

## Apppendix: Pytorch Lightning

- 실무에서는 생산성이 중요하다.
- 지금까지의 모든 프로세스를 하나의 클래스로 정의한다.
- `fit()` 으로 전체 과정을 실행시킨다.
  - Keras 코드를 보는 듯...

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=15Rk8xviskbXlqJGCzqYV5Ql22k_68qWC' width=500/>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1rISeeMHxIANkxw2okp30IEJCN6u6uhop' width=900/>

- 출처: https://www.pytorchlightning.ai/

<br>

- 그래도 공부는 Pytorch로 했으면 해요.
- 충분한 이해가 바탕이 되지 않은 상태에서는 오히려 독이 될 수 있습니다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1Dk5RM2INx4Ftc-3d_DBguU_mzJCrN_ok' width=500/>

- 코드로부터 머신러닝 프로세스를 배울 수 있고 자유롭게 응용할 수 있음